In [1]:
from pytorch_grad_cam import GradCAM, HiResCAM, ScoreCAM, GradCAMPlusPlus, AblationCAM, XGradCAM, EigenCAM, FullGrad
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image
from torchvision.models import resnet50

model = resnet50(pretrained=True)
target_layers = [model.layer4[-1]]
input_tensor = # Create an input tensor image for your model..
# Note: input_tensor can be a batch tensor with several images!

# Construct the CAM object once, and then re-use it on many images:
cam = GradCAM(model=model, target_layers=target_layers)

# You can also use it within a with statement, to make sure it is freed,
# In case you need to re-create it inside an outer loop:
# with GradCAM(model=model, target_layers=target_layers) as cam:
#   ...

# We have to specify the target we want to generate
# the Class Activation Maps for.
# If targets is None, the highest scoring category
# will be used for every image in the batch.
# Here we use ClassifierOutputTarget, but you can define your own custom targets
# That are, for example, combinations of categories, or specific outputs in a non standard model.

targets = [ClassifierOutputTarget(281)]

# You can also pass aug_smooth=True and eigen_smooth=True, to apply smoothing.
grayscale_cam = cam(input_tensor=input_tensor, targets=targets)

# In this example grayscale_cam has only one image in the batch:
grayscale_cam = grayscale_cam[0, :]
visualization = show_cam_on_image(rgb_img, grayscale_cam, use_rgb=True)

# You can also get the model outputs without having to re-inference
model_outputs = cam.outputs

SyntaxError: invalid syntax (3850420894.py, line 8)

In [4]:
import os
import logging
from time import time
import numpy as np
import torch
import pandas as pd
from joblib import Parallel, delayed
from meegnet.parsing import parser
from meegnet.params import TIME_TRIAL_LENGTH
from meegnet.utils import load_checkpoint, cuda_check
from meegnet.network import create_net
from meegnet.dataloaders import load_data
from meegnet.util_viz import (
    get_positive_negative_saliency,
    compute_saliency_based_psd,
)
from pytorch_grad_cam import GuidedBackpropReLUModel
from pytorch_grad_cam import GradCAM, HiResCAM, ScoreCAM, GradCAMPlusPlus, AblationCAM, XGradCAM, EigenCAM, FullGrad
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image

DEVICE = cuda_check()
CHANNELS = ("MAG", "PLANAR1", "PLANAR2")

def compute_saliency_maps(sub, sal_path, GBP, args):
    data, targets = load_data(
        dataframe.loc[dataframe["sub"] == sub],
        args.data_path,
        epoched=args.epoched,
        seed=args.seed,
        s_freq=args.sfreq,
        chan_index=chan_index,
        datatype=args.datatype,
        eventclf=args.eventclf,
    )
    if data is None or targets is None:
        return
    if args.eventclf:
        target_saliencies = [[[], []], [[], []]]
        target_psd = [[[], []], [[], []]]
    else:
        target_saliencies = [[], []]
        target_psd = [[], []]

    # For each of those trial with associated label:
    for trial, label in zip(data, targets):
        X = trial[np.newaxis].type(torch.FloatTensor).to(DEVICE)
        if len(X.shape) < 4:
            X = X[np.newaxis, :]
        # Compute predictions of the trained network, and confidence
        preds = torch.nn.Softmax(dim=1)(net(X)).detach().cpu()
        pred = preds.argmax().item()
        confidence = preds.max()
        if args.subclf:
            label = int(dataframe[dataframe["sub"] == sub].index[0])

        # If the confidence reaches desired treshhold (given by args.confidence)
        if confidence >= args.confidence and pred == label:
            # Compute Guided Back-propagation for given label projected on given data X
            guided_grads = GBP(X.to(DEVICE), label)
            guided_grads = np.rollaxis(guided_grads, 2, 0)
            # Compute saliencies
            pos_saliency, neg_saliency = get_positive_negative_saliency(guided_grads)

            # Depending on the task, add saliencies in lists
            if args.eventclf:
                target_saliencies[label][0].append(pos_saliency)
                target_saliencies[label][1].append(neg_saliency)
                if args.compute_psd:
                    target_psd[label][0].append(
                        compute_saliency_based_psd(
                            pos_saliency, trial, args.w_size, args.sfreq
                        )
                    )
                    target_psd[label][1].append(
                        compute_saliency_based_psd(
                            neg_saliency, trial, args.w_size, args.sfreq
                        )
                    )
            else:
                target_saliencies[0].append(pos_saliency)
                target_saliencies[1].append(neg_saliency)
                if args.compute_psd:
                    target_psd[0].append(
                        compute_saliency_based_psd(
                            pos_saliency, trial, args.w_size, args.sfreq
                        )
                    )
                    target_psd[1].append(
                        compute_saliency_based_psd(
                            neg_saliency, trial, args.w_size, args.sfreq
                        )
                    )
    # With all saliencies computed, we save them in the specified save-path
    for j, sal_type in enumerate(("pos", "neg")):
        if args.eventclf:
            for i, label in enumerate(labels):
                sal_filepath = os.path.join(
                    sal_path,
                    f"{sub}_{labels[i]}_{sal_type}_sal_{args.confidence}confidence.npy",
                )
                np.save(sal_filepath, np.array(target_saliencies[i][j]))
                if args.compute_psd:
                    psd_filepath = os.path.join(
                        psd_path,
                        f"{sub}_{labels[i]}_{sal_type}_psd_{args.confidence}confidence.npy",
                    )
                    np.save(psd_filepath, np.array(target_psd[i][j]))
        else:
            lab = "" if args.subclf else f"_{labels[label]}"
            sal_filepath = os.path.join(
                sal_path,
                f"{sub}{lab}_{sal_type}_sal_{args.confidence}confidence.npy",
            )
            np.save(sal_filepath, np.array(target_saliencies[j]))
            if args.compute_psd:
                lab = "" if args.subclf else f"_{labels[label]}"
                psd_filepath = os.path.join(
                    psd_path,
                    f"{sub}{lab}_{sal_type}_psd_{args.confidence}confidence.npy",
                )
                np.save(psd_filepath, np.array(target_psd[j]))




In [6]:
labels = ["visual", "auditory"]  # image is label 0 and sound label 1
trial_length = TIME_TRIAL_LENGTH
n_channels = 306
chan_index = [0, 1, 2]
input_size = (n_channels // 102, 102, trial_length)
fold = 1
name = f"net_42_fold{fold}_ALL"
suffixes = ""
n_outputs = 2
epoched = True
datatype = "passive"
seed=43
max_subj=300
net_option="best_net"

MODEL_PATH = "/home/arthur/github/meegnet/"
DATA_PATH = "/home/arthur/github/meegnet/data/"


#####################################
### LOADING NETWORK AND DATA INFO ###
#####################################

model_filepath = os.path.join(MODEL_PATH, name + ".pt")
net = create_net(net_option, name, input_size, n_outputs, DEVICE, args)
_, net_state, _ = load_checkpoint(model_filepath)
net.load_state_dict(net_state)

dataframe = (
    pd.read_csv(
        os.path.join(DATA_PATH, f"participants_info_{datatype}.csv"),
        index_col=0,
    )
    .sample(frac=1, random_state=seed)
    .reset_index(drop=True)[:max_subj]
)
subj_list = dataframe["sub"]

#################
### MAIN LOOP ###
#################

target_layers = [net.layer4[-1]]
data, targets = load_data(
        dataframe.loc[dataframe["sub"] == sub],
        DATA_PATH,
        epoched=epoched,
        seed=seed,
        s_freq=500,
        chan_index=chan_index,
        datatype=datatype,
        eventclf=eventclf,
    )
trial, label = next(zip(data, targets))
X = trial[np.newaxis].type(torch.FloatTensor).to(DEVICE)
if len(X.shape) < 4:
    X = X[np.newaxis, :]

input_tensor = X # Create an input tensor image for your model..
# Note: input_tensor can be a batch tensor with several images!

# Construct the CAM object once, and then re-use it on many images:
cam = GradCAM(model=net, target_layers=target_layers, device=DEVICE)

# You can also use it within a with statement, to make sure it is freed,
# In case you need to re-create it inside an outer loop:
# with GradCAM(model=model, target_layers=target_layers) as cam:
#   ...

# We have to specify the target we want to generate
# the Class Activation Maps for.
# If targets is None, the highest scoring category
# will be used for every image in the batch.
# Here we use ClassifierOutputTarget, but you can define your own custom targets
# That are, for example, combinations of categories, or specific outputs in a non standard model.

targets = [label]

# You can also pass aug_smooth=True and eigen_smooth=True, to apply smoothing.
grayscale_cam = cam(input_tensor=input_tensor, targets=targets)

# In this example grayscale_cam has only one image in the batch:
grayscale_cam = grayscale_cam[0, :]
visualization = show_cam_on_image(rgb_img, grayscale_cam, use_rgb=True)

# You can also get the model outputs without having to re-inference
model_outputs = cam.outputs

NameError: name 'args' is not defined